## Module 4. Materialized Views for Performance Tuning 

Module 4 is to focus on exploring and discovering how a query can benefit from Materialized View.

### 4.1 Identify MV Candidates

In this step, we will explore on how to determine if Materialized View can help in improvement performance of our queries. MV is mostly helpful on expensive aggregations. There are many ways to decide on expensive aggregation. Here we use a simple way based on the number of rows of aggregation output for the sake of time. You can also make decision based on spillage depending on your workloads and needs.

In [ ]:
alter session set use_cached_result = false;  -- disable result caching

-- Expensive aggregration with output rows 
SELECT
    query_tag,
    query_id,
    start_time,
    operator_id,
    operator_type,
    operator_statistics:output_rows agg_out_rows,
    operator_attributes,
    operator_attributes:grouping_keys grouping_keys,
    operator_statistics:spilling:bytes_spilled_local_storage as bytes_spilled_local_storage
FROM
    base_query_stats qs  
where 
    operator_type = 'Aggregate'
    and ( 
        grouping_keys ilike '%t.%' or 
        grouping_keys ilike '%traffic.%' 
    )
    and query_tag like 'BASE WORKLOAD QUERY%'
qualify row_number() over (partition by query_tag order by start_time desc) = 1
order by agg_out_rows desc, query_tag
;

We can see that query 09 has the most number of aggregation output, which is considered as a good potential candidate that may benefit from MV.

```json
{
  "functions": [
    "COUNT(DISTINCT TO_CHAR(T.TIMESTAMP, 'YYYYMM'))",
    "SUM(COUNT(T.URL))",
    "COUNT(COUNT(T.URL))"
  ],
  "grouping_keys": [
    "T.USER_ID"
  ]
}
```
### 4.2 Design Materialized View

Let's revisit what query 09 does. 

```sql
WITH user_sessions AS ( -- BASE WORKLOAD QUERY - 09
  SELECT
    user_id,
    TO_CHAR (timestamp, 'YYYYMM') AS visit_month,
    COUNT(*) AS total_visits,
    COUNT(t.url) AS pages_visited
  FROM
    traffic AS t
  WHERE visit_month between '202308' and '202412'
    and user_id BETWEEN 1000000 AND 2000000
  GROUP BY ALL
)
  SELECT
    s.user_id,
    COUNT(DISTINCT visit_month) AS total_visit_month,
    AVG(pages_visited) AS avg_pages_per_session --,
  FROM
    user_sessions s
  GROUP BY
    s.user_id
    ;
```

Based on the expensive aggregation, create a materialized view based on the aggregation in Query 09.

In [ ]:
-- use large warehouse to speed up the MV creation
USE WAREHOUSE WH_SUMMIT25_PERF_OPS;

CREATE OR REPLACE MATERIALIZED VIEW MV_TRAFFIC_AGGREGATION AS
SELECT
  TO_CHAR (timestamp, 'YYYYMM') AS visit_month,
  user_id,
  COUNT(*) AS total_visits,
  COUNT( url) AS unique_pages
FROM
  traffic
WHERE
  user_id BETWEEN 1000000 AND 2000000
GROUP BY 1,2;

In [ ]:
-- Check on behind_by column on how far the MV refresh is behind by 
SHOW MATERIALIZED VIEWS LIKE 'MV_TRAFFIC_AGGREGATION';

### 4.3 Rerun Query on WH_SUMMIT25_PERF_MV

We need to warm the new warehouse WH_SUMMIT25_PERF_MV first by running query 01 and 02 similar to base workload on WH_SUMMIT25_PERF_BASE.

Then Rerun the query 09 on MV warehouse WH_SUMMIT25_PERF_MV, 

In [ ]:
-- Switch to warehouse WH_SUMMIT25_PERF_MV for easier performance and cost comparison
USE WAREHOUSE WH_SUMMIT25_PERF_MV; 

-- rerun query 01 and 02 to warm the new warehouse WH_SUMMIT25_PERF_MV
with age_20_to_30 as ( -- BASE WORKLOAD QUERY - 01
    select distinct uuid
    from user_profile
    where question_id = 3 -- DOB question
        and value::date between dateadd(year, -30, current_date) 
            and dateadd(year, -20, current_date)
),
gender_male as (
    select distinct uuid
    from user_profile
    where question_id = 4 -- Gender question
        and value::string = 'M'
),
income_50K_to_100K as (
    select distinct uuid
    from user_profile
    where question_id = 10 -- Income question
        and value::int between 50000 and 100000
)
select
    c.name,
    url,
    count(1) as visits
from traffic t
join category c on (
    c.id = t.category_id
)
join age_20_to_30 a on (
    a.uuid = t.uuid
)
join gender_male g on (
    g.uuid = t.uuid
)
join income_50K_to_100K i on (
    i.uuid = t.uuid
)
where
    t.timestamp between '2025-01-01' and '2025-02-01'
group by all
qualify row_number() over (
    partition by c.name order by visits desc
) <= 100
order by c.name, visits desc;

WITH url_stats AS ( -- BASE WORKLOAD QUERY - 02
  SELECT
    c.name AS category_name,
    t.url,
    COUNT(DISTINCT t.uuid) AS unique_visitors,
    COUNT(*) AS total_visits,
    RANK() OVER (
      PARTITION BY c.name
      ORDER BY
        COUNT(*) DESC
    ) AS rank_in_category
  FROM
    traffic AS t
    JOIN category AS c ON t.category_id = c.id
  WHERE
    t.timestamp between '2025-01-01' and '2025-02-01'
  GROUP BY
    c.name,
    t.url
)
SELECT
  category_name,
  url,
  unique_visitors,
  total_visits,
  rank_in_category
FROM
  url_stats
WHERE
  rank_in_category <= 10
ORDER BY
  category_name,
  rank_in_category;
  
-- rerun query 09 for MV 
WITH user_sessions AS ( -- MV WORKLOAD QUERY - 09
  SELECT
    user_id,
    TO_CHAR (timestamp, 'YYYYMM') AS visit_month,
    COUNT(*) AS total_visits,
    COUNT(t.url) AS pages_visited
  FROM
    traffic AS t
  WHERE visit_month between '202308' and '202412'
    and user_id BETWEEN 1000000 AND 2000000
  GROUP BY ALL
)
  SELECT
    s.user_id,
    COUNT(DISTINCT visit_month) AS total_visit_month,
    AVG(pages_visited) AS avg_pages_per_session 
  FROM
    user_sessions s
  GROUP BY
    s.user_id
    ;

After rerun the above query 09, Go to query history and identify the rerun query, and check query profile and see if the MV has been used. Why or Why Not?

( Note: Check the screenshots in the guide if you need more instructions on how to get to the query profile. )

### 4.4 Compare performance

In this step, we will collect performance metrics and compare performance of MV optimization.

In [ ]:
USE WAREHOUSE WH_SUMMIT25_PERF_OPS;

-- compare performance
select 
    warehouse_name,
    REGEXP_SUBSTR(query_text, 'BASE WORKLOAD QUERY - [0-9]{2}') as my_tag,
    total_elapsed_time
FROM TABLE(INFORMATION_SCHEMA.QUERY_HISTORY(RESULT_LIMIT =>10000))   
where 
    execution_time > 0
    and warehouse_name = 'WH_SUMMIT25_PERF_BASE'
    and error_code is null 
    and query_type = 'SELECT'
    and query_text ilike '%BASE WORKLOAD QUERY - 09%'
qualify row_number() over (partition by my_tag order by start_time desc) = 1

union all

select 
    warehouse_name,
    REGEXP_SUBSTR(query_text, 'MV WORKLOAD QUERY - [0-9]{2}') as my_tag,
    total_elapsed_time
FROM TABLE(INFORMATION_SCHEMA.QUERY_HISTORY(RESULT_LIMIT =>10000))   
where 
    execution_time > 0
    and warehouse_name = 'WH_SUMMIT25_PERF_MV'
    and error_code is null 
    and query_type = 'SELECT'
    and query_text ilike '%MV WORKLOAD QUERY - 09%'
qualify row_number() over (partition by my_tag order by start_time desc) = 1
;

### 4.5 Compare Costs

In this step, we will compare Costs. 
- Compare the cost metrics of `WH_SUMMIT25_PERF_BASE` and `WH_SUMMIT25_PERF_MV`. 
- Discover the benefits of the materialized view (reduced execution time) and its associated storage costs.

In [ ]:
-- comparing cost, note WH_SUMMIT25_PERF_BASE contains more other queries
select 
    WAREHOUSE_NAME,
    SUM(CREDITS_USED)
from table(information_schema.warehouse_metering_history(
    dateadd('days', -10, current_date())
))
where WAREHOUSE_NAME in ( 'WH_SUMMIT25_PERF_BASE', 'WH_SUMMIT25_PERF_MV')
group by 1;

You can also use the following query to check the MV refresh history and its associated costs. However, due to latency or minimal cost (i.e., less than 0.01), you might not be able to see it in this lab.

In [ ]:
-- MV refresh cost, there might be latency  
select 
    *
from table(
    information_schema.materialized_view_refresh_history(
        date_range_start => dateadd(h, -10, current_date),
        materialized_view_name => 'SQL_PERF_OPTIMIZATION.PUBLIC.MV_TRAFFIC_AGGREGATION'
    )
)
;